# Notebook 3: Cox Proportional Hazards Regression

## Model Overview

Cox PH is a semi-parametric survival model:
- **Assumption**: Hazards are proportional (h(t|X) = h_0(t)*exp(beta^T X))
- **Output**: Linear risk score (higher = worse prognosis)
- **Regularization**: Elastic net (L1 + L2) via `CoxnetSurvivalAnalysis`

## Configuration
- **Features**: 128 fixed (scaled)
- **Regularization**: Elastic net with tuned `l1_ratio` and `alpha`
- **Evaluation**: `concordance_index_ipcw` from sksurv (competition metric)
- **CV Score**: See results below

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_ipcw
from sksurv.util import Surv
import optuna
from optuna.samplers import TPESampler

TRAIN_PATH = '/your_path/SurvivalPrediction/data'

## 1. Load Data

In [18]:
# Load 128-feature FIXED dataset
X_train_full = pd.read_csv(f'{TRAIN_PATH}/X_train_128features_clean_fixed.csv')
target = pd.read_csv(f'{TRAIN_PATH}/target_train_clean_aligned.csv')

# Align if needed
if 'ID' in X_train_full.columns:
    valid_ids = set(target['ID'].values)
    mask = X_train_full['ID'].isin(valid_ids)
    X_train_full = X_train_full[mask].copy()
    X_train_full = X_train_full.set_index('ID').loc[target['ID']].reset_index()
    X_train = X_train_full.drop(columns=['ID'])
else:
    X_train = X_train_full.copy()

y_time = target['OS_YEARS'].values
y_event = target['OS_STATUS'].values.astype(bool)
n_samples = len(X_train)

# Create structured array for sksurv
y_surv = Surv.from_arrays(event=y_event, time=y_time)

print(f"Features: {X_train.shape[1]}")
print(f"Samples: {n_samples}")
print(f"Events: {y_event.sum()} ({y_event.mean()*100:.1f}%)")

Features: 128
Samples: 3120
Events: 1600 (51.3%)


In [19]:
# Risk groups for weighted C-index
def define_risk_groups(X):
    risk_factors = pd.DataFrame(index=X.index)
    risk_factors['high_blast'] = (X['BM_BLAST'] > 10).astype(int)
    risk_factors['has_TP53'] = (X['has_TP53'] > 0).astype(int)
    risk_factors['low_hb'] = (X['HB'] < 10).astype(int)
    risk_factors['low_plt'] = (X['PLT'] < 50).astype(int)
    risk_factors['high_cyto'] = (X['cyto_risk_score'] >= 3).astype(int)
    n_risk_factors = risk_factors.sum(axis=1)
    return {
        'test_like': n_risk_factors >= 1,
        'high_risk': n_risk_factors >= 2,
    }

risk_groups = define_risk_groups(X_train)

# Stratification variable
has_tp53 = (X_train['has_TP53'] > 0).astype(int).values
strat_var = pd.Series([f"{int(e)}_{int(t)}" for e, t in zip(y_event, has_tp53)])

## 2. Evaluation Metric

Use `concordance_index_ipcw` from scikit-survival:
- IPCW (Inverse Probability of Censoring Weighting) handles censoring properly
- Time truncation at τ = 7.0 years

Weighted formula: 0.3 × overall + 0.4 × test_like + 0.3 × high_risk

In [1]:
def weighted_cindex_ipcw(risk, y_surv_all, risk_groups, tau=7.0):
    """Compute weighted C-index using concordance_index_ipcw (competition metric)."""
    
    c_overall = concordance_index_ipcw(y_surv_all, y_surv_all, risk, tau=tau)[0]

    mask_test = risk_groups['test_like'].values 
    y_surv_test = Surv.from_arrays(event=y_surv_all['event'][mask_test], time=y_surv_all['time'][mask_test])
    c_test = concordance_index_ipcw(y_surv_all, y_surv_test, risk[mask_test], tau=tau)[0]

    mask_high = risk_groups['high_risk'].values
    y_surv_high = Surv.from_arrays(event=y_surv_all['event'][mask_high], time=y_surv_all['time'][mask_high])
    c_high = concordance_index_ipcw(y_surv_all, y_surv_high, risk[mask_high], tau=tau)[0]

    weighted = 0.3 * c_overall + 0.4 * c_test + 0.3 * c_high
    return {'overall': c_overall, 'test_like': c_test, 'high_risk': c_high, 'weighted': weighted}

## 3. Global OOF Evaluation

In [21]:
def global_oof_evaluate(l1_ratio, alpha, n_splits=5, seed=42):
    """Global OOF Cross-validation for CoxPH with elastic net."""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof_preds = np.zeros(n_samples)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train, strat_var)):
        X_tr = X_train.iloc[train_idx]
        X_val = X_train.iloc[val_idx]
        y_surv_tr = Surv.from_arrays(event=y_event[train_idx], time=y_time[train_idx])
        
        # Scale
        scaler = StandardScaler()
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_val_scaled = scaler.transform(X_val)
        
        # Train CoxnetSurvivalAnalysis (fits full alpha path)
        model = CoxnetSurvivalAnalysis(l1_ratio=l1_ratio, alphas=[alpha])
        model.fit(X_tr_scaled, y_surv_tr)
        oof_preds[val_idx] = model.predict(X_val_scaled)
    
    # Global Z-score normalization
    oof_normalized = (oof_preds - oof_preds.mean()) / (oof_preds.std() + 1e-8)
    
    return weighted_cindex_ipcw(oof_normalized, y_surv, risk_groups)

## 4. Hyperparameter Tuning

In [22]:
def objective(trial):
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
    alpha = trial.suggest_float('alpha', 0.0001, 50.0, log=True)
    result = global_oof_evaluate(l1_ratio, alpha)
    return result['weighted']

print("Running Optuna hyperparameter tuning (elastic net, 200 trials)...")
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=200, show_progress_bar=True)

print(f"\nBest l1_ratio: {study.best_params['l1_ratio']:.6f}")
print(f"Best alpha: {study.best_params['alpha']:.6f}")
print(f"Best weighted C-index: {study.best_value:.4f}")

# Show top 10 trials
trials_df = study.trials_dataframe().sort_values('value', ascending=False).head(10)
print("\nTop 10 trials:")
print(trials_df[['number', 'value', 'params_l1_ratio', 'params_alpha']])

Running Optuna hyperparameter tuning (elastic net, 200 trials)...


  0%|          | 0/200 [00:00<?, ?it/s]


Best l1_ratio: 0.437979
Best alpha: 0.039145
Best weighted C-index: 0.6907

Top 10 trials:
     number     value  params_l1_ratio  params_alpha
191     191  0.690720         0.437979      0.039145
105     105  0.690707         0.542419      0.032105
108     108  0.690702         0.514335      0.033556
141     141  0.690692         0.510778      0.034296
162     162  0.690692         0.488870      0.034848
113     113  0.690677         0.438772      0.037749
173     173  0.690670         0.438410      0.040900
111     111  0.690660         0.526913      0.033445
172     172  0.690659         0.462981      0.038304
199     199  0.690656         0.369403      0.049073


## 5. Best Model Results

In [23]:
# Use best params from Optuna
BEST_L1_RATIO = study.best_params['l1_ratio']
BEST_ALPHA = study.best_params['alpha']

result = global_oof_evaluate(BEST_L1_RATIO, BEST_ALPHA)
print("CoxPH Elastic Net Results (concordance_index_ipcw):")
print(f"  L1 Ratio: {BEST_L1_RATIO:.4f}")
print(f"  Alpha: {BEST_ALPHA:.4f}")
print(f"  Overall C-index: {result['overall']:.4f}")
print(f"  Test-like C-index: {result['test_like']:.4f}")
print(f"  High-risk C-index: {result['high_risk']:.4f}")
print(f"  Weighted C-index: {result['weighted']:.4f}")

CoxPH Elastic Net Results (concordance_index_ipcw):
  L1 Ratio: 0.4380
  Alpha: 0.0391
  Overall C-index: 0.7175
  Test-like C-index: 0.6931
  High-risk C-index: 0.6607
  Weighted C-index: 0.6907


## 6. Train Final Model and Generate Predictions

In [24]:
# Scale all training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train final model with elastic net
final_model = CoxnetSurvivalAnalysis(l1_ratio=BEST_L1_RATIO, alphas=[BEST_ALPHA])
final_model.fit(X_train_scaled, y_surv)

# Show sparsity (number of zero coefficients from L1)
n_zero = (final_model.coef_.ravel() == 0).sum()
print(f"Model trained with l1_ratio={BEST_L1_RATIO:.4f}, alpha={BEST_ALPHA:.4f}")
print(f"Non-zero coefficients: {X_train.shape[1] - n_zero}/{X_train.shape[1]} (L1 zeroed out {n_zero} features)")

Model trained with l1_ratio=0.4380, alpha=0.0391
Non-zero coefficients: 61/128 (L1 zeroed out 67 features)


In [25]:
# Load test data
X_test_full = pd.read_csv(f'{TRAIN_PATH}/X_test_128features_fixed_scaled.csv')
X_test_with_id = pd.read_csv(f'{TRAIN_PATH}/X_test_83features_with_id.csv')
test_ids = X_test_with_id['ID'].values

# Predict
test_risk = final_model.predict(X_test_full.values)

print(f"Test predictions: {len(test_risk)} samples")
print(f"Risk range: [{test_risk.min():.4f}, {test_risk.max():.4f}]")

# Save submission
submission = pd.DataFrame({'ID': test_ids, 'risk_score': test_risk})
submission.to_csv(f'{TRAIN_PATH}/submission_coxph_128features.csv', index=False)
print(f"Saved: submission_coxph_128features.csv")

Test predictions: 1193 samples
Risk range: [-1.8615, 4.9928]
Saved: submission_coxph_128features.csv


## Summary

### CoxPH Elastic Net Results (Competition Metric)

Results from `CoxnetSurvivalAnalysis` with tuned `l1_ratio` and `alpha` (see output above).

### Key Findings
1. Elastic net (L1 + L2) allows automatic feature selection via L1 sparsity
2. Tuning both `l1_ratio` and `alpha` via Optuna with global OOF evaluation
3. L1 component can zero out irrelevant features from the 128-feature set
4. Useful for ensemble diversity alongside tree-based models